In [1]:
![ ! -f requirements.txt ] && git clone https://github.com/anudaweerasinghe/small-coder.git
%cd /content/small-coder

Cloning into 'small-coder'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 28 (delta 4), reused 17 (delta 1), pack-reused 0
Receiving objects: 100% (28/28), 40.87 KiB | 6.81 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
%%capture

! pip3 install -r requirements.txt

In [3]:
!git clone https://github.com/bigcode-project/bigcode-evaluation-harness.git
%cd bigcode-evaluation-harness

Cloning into 'bigcode-evaluation-harness'...
remote: Enumerating objects: 4107, done.
remote: Counting objects: 100% (1689/1689), done.
remote: Compressing objects: 100% (487/487), done.
remote: Total 4107 (delta 1360), reused 1381 (delta 1193), pack-reused 2418
Receiving objects: 100% (4107/4107), 818.20 KiB | 20.98 MiB/s, done.
Resolving deltas: 100% (2794/2794), done.
/content/bigcode-evaluation-harness


In [4]:
%%capture

!pip install -e .

Obtaining file:///content/bigcode-evaluation-harness
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupt

In [7]:
!accelerate config default
!mv /root/.cache/huggingface/accelerate/default_config.yaml /content/

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [9]:
!accelerate launch  main.py \
  --model "deepseek-ai/deepseek-coder-1.3b-instruct" \
  --max_length_generation 1024 \
  --tasks humaneval \
  --temperature 0.2 \
  --n_samples 1 \
  --batch_size 1 \
  --allow_code_execution

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2024-04-13 00:21:45.402544: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 00:21:45.402590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 00:21:45.403895: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alread

In [ ]:
!accelerate launch  main.py \
  --model "deepseek-ai/deepseek-coder-1.3b-instruct" \
  --max_length_generation 1024 \
  --tasks mbpp \
  --temperature 0.2 \
  --n_samples 1 \
  --batch_size 1 \
  --allow_code_execution

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2024-04-13 00:35:32.025759: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 00:35:32.025810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 00:35:32.027219: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alread